In [1]:
import os, math
import pandas as pd
import numpy as np

In [2]:
from gensim.models import Word2Vec as w2v
from os import path as osp

root = osp.dirname(os.getcwd())
model = w2v.load(osp.join(root,'model/stackexchange_model'))


def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [3]:
cvs = pd.read_csv('prc_data.csv', sep='\t')
cvs = cvs.set_index('Unnamed: 0')

prc_description = "ne datum analyst knowledge python R"

In [4]:
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        print similar_words[i], word_value[similar_words[i]] 

ne 1
je 0.636240243912
ke 0.629526257515
datum 1
datums 0.602979719639
wgs 0.574095129967
analyst 1
analysts 0.489208996296
engineer 0.480672895908
knowledge 1
expertise 0.707411766052
knowlege 0.643639862537
python 1
perl 0.720492959023
ruby 0.67925286293
R 1
URI 0.981574892998
L 0.977639019489


In [9]:
no_of_cv = 100

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            if word in cvs.loc(0)['skill'][i].split() or word in cvs.loc(0)['exp'][i].split():
                count[word] += 1
        except:
            pass
    if (count[word] == 0):
        count[word] = 1
    idf[word] = math.log(no_of_cv/count[word])
print count
idf

{u'knowlege': 1, 'knowledge': 27, u'ke': 1, 'python': 1, 'datum': 9, u'L': 1, u'datums': 1, 'ne': 3, 'R': 1, u'expertise': 5, u'analysts': 1, u'wgs': 1, u'je': 1, u'perl': 1, u'ruby': 3, 'analyst': 4, u'URI': 1, u'engineer': 17}


{u'L': 4.605170185988092,
 'R': 4.605170185988092,
 u'URI': 4.605170185988092,
 'analyst': 3.2188758248682006,
 u'analysts': 4.605170185988092,
 'datum': 2.3978952727983707,
 u'datums': 4.605170185988092,
 u'engineer': 1.6094379124341003,
 u'expertise': 2.995732273553991,
 u'je': 4.605170185988092,
 u'ke': 4.605170185988092,
 'knowledge': 1.0986122886681098,
 u'knowlege': 4.605170185988092,
 'ne': 3.4965075614664802,
 u'perl': 4.605170185988092,
 'python': 4.605170185988092,
 u'ruby': 3.4965075614664802,
 u'wgs': 4.605170185988092}

In [10]:
score = {}
for i in range(no_of_cv):
    score[i] = 0
    try:
        for word in word_value.keys():
            tf = cvs.loc(0)['skill'][i].split().count(word) + cvs.loc(0)['exp'][i].split().count(word)
            score[i] += word_value[word]*tf*idf[word]
    except:
        pass

In [11]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], i))
    
sorted_list.sort(reverse \
                 = True)

for s, i in sorted_list:
    print list(cvs)[i], ':', s

cv174 : 18.4263252014
cv107 : 10.1324929497
cv126 : 6.99301512293
cv110 : 6.43775164974
cv129 : 5.84863783144
cv141 : 5.6937321388
cv150 : 5.61677109767
cv123 : 5.29072471321
cv175 : 4.75002554277
cv135 : 4.64167909293
cv122 : 4.64167909293
cv109 : 4.64167909293
cv124 : 4.51711153105
cv142 : 3.99248900702
cv119 : 3.94512163711
cv164 : 3.49650756147
cv172 : 3.47362506005
cv149 : 3.295836866
cv159 : 3.21887582487
cv16 : 3.21782854692
cv148 : 3.21782854692
cv111 : 2.97083775949
cv102 : 2.64583865298
cv139 : 2.3978952728
cv177 : 2.19722457734
cv105 : 2.11921625825
cv120 : 1.87222547082
cv1.txt : 1.87222547082
cv187 : 1.54722636431
cv113 : 1.54722636431
cv176 : 1.09861228867
cv17 : 1.09861228867
cv169 : 1.09861228867
cv163 : 1.09861228867
cv156 : 1.09861228867
cv155 : 1.09861228867
cv153 : 1.09861228867
cv145 : 1.09861228867
cv127 : 1.09861228867
cv121 : 1.09861228867
cv116 : 1.09861228867
cv108 : 1.09861228867
cv106 : 1.09861228867
cv184 : 0.773613182154
cv18 : 0.773613182154
cv15 : 0.7736